## PageRank Lab

### Problem 1:

In [108]:
import numpy as np
from scipy import linalg as la

def get_A():
    A = np.zeros((8,8))
    with open('matrix.txt', 'r') as myfile:
        for line in myfile:
            line = line.strip().split()
            try:
                line = list(map(int, line))
                n_in = line[0]
                n_out = line[1]
                A[n_in, n_out] =  1
            except:
                continue
    return A

In [109]:
get_A()

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

### Problem 2:

In [214]:
def get_K(A):
    # make modified adjacency matrix
    r_sum = np.sum(A, axis=1)
    Am = np.copy(A)
    N = np.shape(Am)[0]
    for i in range(N):
        if r_sum[i] == 0:
            Am[i,:] = 1

    # make diagonal matrix
    D = np.zeros(N)
    r_sum = np.sum(Am, axis=1)
    for i in range(N):
        D[i] = r_sum[i]

    # compute K with array broadcasting
    K = np.zeros_like(A)
    for i in range(N):
        if D[i] != 0:
            K = Am / D[:,None]
            K = K.T

    return K

### Problem 3:

In [230]:
def get_SS(A, N=None, d=0.85, tol=1e-5):
    if N != None:
        A = A[:N,:N]
    else: 
        N = np.shape(A)[0]
    K = get_K(A)
    p = np.random.rand(N)
    while True:
        p_next = d * (K @ p) + ((1 - d) / N) * np.ones(N)
        if np.allclose(p_next, p, tol):
            break
        p = p_next
    
    return p
#    print(p)

In [231]:
get_SS(get_A())

array([ 0.43870075,  0.02171029,  0.02786154,  0.02171029,  0.02171029,
        0.02786154,  0.04585394,  0.39460421])

### Problem 4:

In [232]:
def get_SS_ev(A, N=None, d=0.85):
    if N != None:
        A = A[:N,:N]
    else:
        N = np.shape(A)[0]
    K = get_K(A)
    B = d * K + ((1 - d) / N) * np.ones(N)
    eigs = la.eig(B)
    evals = eigs[0]
    evecs = eigs[1]
    
    return evecs[:,0] / sum(evecs[:,0])

In [233]:
get_SS_ev(get_A())

array([ 0.43869288,  0.02171029,  0.02786154,  0.02171029,  0.02171029,
        0.02786154,  0.04585394,  0.39459924])

### Problem 5:

In [260]:
def rank_bball():
    teams = set()
    with open('ncaa2013.csv', 'r') as ncaafile:
        ncaafile.readline() # skip first line
        for line in ncaafile:
            match = line.strip().split(',')
            teams.add(match[0])
            teams.add(match[1])
    
    teams = np.array(list(teams))
    N = np.size(teams)
    A = np.zeros((N,N))
    index_dict = dict(enumerate(teams))
    index_dict = {v: k for k, v in index_dict.items()}
    
    A = np.zeros((N,N))
    with open('ncaa2013.csv', 'r') as ncaafile:
        ncaafile.readline()
        for line in ncaafile:
            match = line.strip().split(',')
            winner = index_dict[match[0]]
            loser = index_dict[match[1]]
            A[loser, winner] = 1
      
    rank = get_SS(A, d=0.7)
    rank_sort = np.argsort(rank)
    teams_sort = teams[rank_sort[::-1]]
    print("The top five teams are:", teams_sort[:5])
        

In [261]:
rank_bball()

The top five teams are: ['Duke' 'Butler' 'Louisville' 'Illinois' 'Indiana']
